### 학습목표
- 멜론 페이지에서 정보를 받아와 top100 데이터 수집
- 노래제목, 가수명 
- 추출한 데이터를 활용 

In [6]:
# 라이브러리 불러오기
import requests as req
from bs4 import BeautifulSoup as bs

In [9]:
# 페이저 정보 받아오기 (res)
url = "https://www.melon.com/chart/"
res = req.get(url)
req.get(url)
# Response[406] : 클라이언트의 요청에 문제 발생 ! 
# => 멜론 서버에서 저작권 문제로 브라우저를 통해 접근하지 않은 것을 인지하고 차단 
# => 요청 시 브라우저 정보를 포함하여 요청 (브라우저 인 "척")
# => 개발자도구 - Network - 헤더정보(user-agent) 필요 
# {'key':'value'}
h = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
res = req.get(url, headers = h)
res

<Response [200]>

In [ ]:
# bs객체화 
soup = bs(res.text, 'lxml')
soup 

- 노래 제목 추출

In [32]:
soup.select('div.ellipsis rank01>span>a')
# 태그 선택 결과 > 빈 리스트 
# -> 왜? 클래스명 사이에 공백 때문 
# 자손 선택자의 기호 ' ' > 따라서 아래와 같이 수정 
song = soup.select('div.ellipsis.rank01>span>a')

In [19]:
# 확인
song[0].text

'Love wins all'

In [ ]:
# 100개 데이터 콘텐츠만 추출 
for i in range(len(song)):
    print(song[i].text)

In [44]:
singer = soup.select('div.ellipsis.rank02')
soup.select('div.ellipsis.rank02')[0].text
# 개수확인
len(singer)
# top100이라고 해서 100개인 줄 알았더니 결과로는 103개 ! 
# => 원하는 정보를 정확하게 가져왔는지 확인해주는 작업 반드시 필요 
# 콘텐츠 확인
singer = soup.select('div.ellipsis.rank02>span')

In [ ]:
for i in range(len(singer)):
    print(singer[i].text)

- 데이터 내보내기 
    1. 분산되어 있는 데이터를 데이터 리스트에 담아주기 
    2. 표와 같은 형태로 만들어주기 (DataFrame화)
    3. csv 파일로 내보내기

In [51]:
# 데이터 리스트에 담기 
# 빈 리스트 생성 
song_list = []
singer_list = []
rank_list = []
# 반복문을 통해 데이터 누적 
for i in range(len(song)):
    song_list.append(song[i].text)
    singer_list.append(singer[i].text)
    rank_list.append(i+1)

In [52]:
display(song_list)

['Love wins all',
 '밤양갱',
 '첫 만남은 계획대로 되지 않아',
 'To. X',
 '비의 랩소디',
 'Love 119',
 '에피소드',
 'Perfect Night',
 '그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))',
 'Drama',
 'EASY',
 'Super Lady',
 '헤어지자 말해요',
 'Wife',
 '사랑은 늘 도망가',
 '인사',
 '꿈',
 'Seven (feat. Latto) - Clean Ver.',
 '모래 알갱이',
 'MANIAC',
 'I AM',
 '우리들의 블루스',
 'Get A Guitar',
 'Do or Die',
 '다시 만날 수 있을까',
 '너의 모든 순간',
 'Baddie',
 'Love Lee',
 '이제 나만 믿어요',
 '무지개',
 'You & Me',
 '사막에서 꽃을 피우듯',
 '예뻤어',
 'London Boy',
 'Polaroid',
 '아버지',
 '잘 지내자, 우리 (여름날 우리 X 로이킴)',
 'Super Shy',
 'Hype Boy',
 '퀸카 (Queencard)',
 '인생찬가',
 '사랑인가 봐',
 '한 페이지가 될 수 있게',
 'Ditto',
 '후라이의 꿈',
 'A bientot',
 'Standing Next to You',
 'ETA',
 '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지',
 '사랑할 수밖에',
 '사건의 지평선',
 '기억해줘요 내 모든 날과 그때를',
 '사랑하지 않아서 그랬어',
 'Spicy',
 '취중고백',
 '심(心)',
 '첫 눈',
 '연애편지',
 '그대가 내 안에 박혔다(그내박)',
 '모든 날, 모든 순간 (Every day, Every Moment)',
 '별 떨어진다 (I Do)',
 '이브, 프시케 그리고 푸른 수염의 아내',
 '봄날',
 '나는 아픈 건 딱 질색이니까',
 'Yes or No (Feat. 허윤진 of LE SSERAFIM, Crush)',


In [48]:
import pandas as pd

In [60]:
# DataFrame화 (dictionary를 활용)
# 키값 -> 컬럼명 
dic = {'노래제목':song_list,
       '가수':singer_list,
       '순위':rank_list}
melon = pd.DataFrame(dic)
melon.set_index('순위', inplace=True)

In [61]:
melon

,노래제목,가수
순위,,
1,Love wins all,아이유
2,밤양갱,비비 (BIBI)
3,첫 만남은 계획대로 되지 않아,TWS (투어스)
4,To. X,태연 (TAEYEON)
5,비의 랩소디,임재현
...,...,...
96,Merry PLLIstmas,PLAVE
97,여섯 번째 여름,PLAVE
98,눈이 오잖아(Feat.헤이즈),이무진


In [63]:
# csv 파일로 내보내기! 
# df명.to_csv('경로/파일명.csb'
melon.to_csv('멜론차트.csv', encoding='euc-kr')
# 인코딩 작업도 진행해주는 것이 좋음 
